In [1]:
from torchvision.io import read_image
from torchvision.transforms import Resize

import torch as tn
import torch.nn as nn
import torchtt as tntt

from torch import optim
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import random_split
import numpy as np

device = tn.device('cuda' if tn.cuda.is_available() else 'cpu')

/home/usr/myenv/lib/python3.12/site-packages/torchtt/_dmrg.py:19: UserWarning: 
C++ implementation not available. Using pure Python.

  warnings.warn("\x1B\nC++ implementation not available. Using pure Python.\n\033")
/home/usr/myenv/lib/python3.12/site-packages/torchtt/_amen.py:21: UserWarning: 
C++ implementation not available. Using pure Python.

  warnings.warn(
/home/usr/myenv/lib/python3.12/site-packages/torchtt/solvers.py:21: UserWarning: 
C++ implementation not available. Using pure Python.

  warnings.warn(
/home/usr/myenv/lib/python3.12/site-packages/torchtt/cpp.py:12: UserWarning: 
C++ implementation not available. Using pure Python.

  warnings.warn("\x1B\nC++ implementation not available. Using pure Python.\n\033")
/home/usr/myenv/lib/python3.12/site-packages/torchtt/__init__.py:34: UserWarning: 
C++ implementation not available. Using pure Python.

  warnings.warn(


In [ ]:
path1 = "data/1_1.jpg"
path2 = "data/1_2.jpg"
path3 = "data/1_3.jpg"
res = Resize((256, 256))

im1 = res(read_image(path1)/255.0)
im2 = res(read_image(path2)/255.0)
im3 = res(read_image(path3)/255.0)

In [14]:
im1_tt = tntt.TT(im1)
im2_tt = tntt.TT(im2)
im3_tt = tntt.TT(im3)

In [15]:
test2 = tntt.kron(im1_tt, im2_tt)
test3 = tntt.kron(test2, im3_tt)

In [34]:
test3

TT with sizes and ranks:
N = [3, 256, 256, 3, 256, 256, 3, 256, 256]
R = [1, 3, 256, 1, 3, 256, 1, 3, 256, 1]

Device: cpu, dtype: torch.float32
#entries 786459 compression 1.034838387820653e-10

In [2]:
class BasicTT(nn.Module):
    def __init__(self):
        super().__init__()
        self.ttl1 = tntt.nn.LinearLayerTT([1, 500, 500, 500], 
                                          [8, 100, 100, 100],
                                          [1, 2  , 2  , 2  , 1])
        
        self.ttl2 = tntt.nn.LinearLayerTT([8, 100, 100, 100],
                                          [8, 50 , 50 , 50 ], 
                                          [1, 5 , 5 , 5 , 1])

        self.ttl3 = tntt.nn.LinearLayerTT([8, 50, 50, 50],
                                          [2, 8, 8, 8],
                                          [1, 10, 10, 10, 1])
        
        self.linear1 = nn.Linear(1024, 512, dtype = tn.float32)

        self.linear2 = nn.Linear(512, 10, dtype = tn.float32)
        self.logsoftmax = nn.LogSoftmax(1)

    def forward(self, x):
        x = self.ttl1(x)
        x = tn.relu(x)
        x = self.ttl2(x)
        x = tn.relu(x)
        x = self.ttl3(x)
        x = tn.relu(x)
        
        x = x.view(-1,1024)
        x = self.linear1(x)
        x = tn.relu(x)
        x = self.linear2(x)
        return self.logsoftmax(x)

In [3]:
model = BasicTT()#.to(device)
loss_function = nn.CrossEntropyLoss()   
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [4]:
test_inp = tn.rand((1,500,500,500))#.cuda()
target = tn.rand((1,10))#.cuda()

optimizer.zero_grad()
output = model(test_inp)
loss = loss_function(output, target)
loss.backward()
optimizer.step()